In [1]:
import requests
import sys
from bs4 import BeautifulSoup
import re
import lxml
from urllib.parse import unquote
import numpy as np
import editdistance
from nltk.corpus import wordnet as wn
from gensim.models.wrappers.fasttext import FastText

model = FastText.load_fasttext_format('./wiki.en')

In [2]:
wiki_prefix = 'https://en.wikipedia.org/wiki/'
title_ignores = ['File:', 'Template:', 'Template_talk:', 'Special:', 'Wikipedia:', 'Portal:', 'Help:',
           'Category:', 'Main_Page', 'Talk:', 'wiktionary', 'Template talk:']
url_ignores = ['wiktionary', 'action=edit', 'wikimedia', 'File:', 'wikidata', 'wikibooks', 'http']

def path_sim(w1, w2):
    s1 = wn.synsets(w1)[0]
    s2 = wn.synsets(w2)[0]
    return s1.path_similarity(s2)

similarity_dict = {
    'fasttext': model.similarity,
    'wordnet': path_sim,
}

present_dict = {
    'fasttext': lambda w: w in model,
    'wordnet': lambda w: len(wn.synsets(w)) > 0,
}

In [3]:
def get_titles2hrefs(target):
    url = wiki_prefix + target
    res = requests.get(url)
    soup = BeautifulSoup(res.text, 'html.parser')
    body = soup.find('div', {'class': 'mw-parser-output'})
    links = body.findAll('a', title=True)
    titles2hrefs = dict()
    for link in links:
        href = link.get('href')
        title = link.get('title')
        if any([i in href for i in url_ignores]):
            continue
        if any([i in title for i in title_ignores]):
            continue
        title = unquote(title).lower()
        if title not in titles2hrefs:
            titles2hrefs[title] = unquote(href[len('/wiki/'):])
    return titles2hrefs

def get_hrefs(target):
    return list(get_titles2hrefs(target).values())

def run_one(start, target, similarity, top):
    assert top >= 1
    present = present_dict[similarity]
    now = start
    path = []
    path.append(now)
    target_s = re.split(',|:|[0-9]|\.| |_|\(|\)', target)
    while target not in [w.lower() for w in [now, '_'.join(now.split(' ')), ' '.join(now.split('_'))]]:
        hrefs = get_hrefs(now)
        hrefs_good = []
        sims = []
        for href in hrefs:
            href_s = re.split(',|:|[0-9]|\.| |_|\(|\)', href.lower())
            sim = 0.0
            href_s = [t for t in href_s if t != '' and present(t)]
            cnt = len(href_s)
            similarities = [similarity_dict[similarity](target_tok, href_tok)
                            for href_tok in href_s for target_tok in target_s]
            similarities = [s for s in similarities if s is not None]
            if len(similarities) > 0 and href not in path:
                sim = np.average(similarities)
                hrefs_good.append(href)
                sims.append(sim)
        argsort = np.argsort(sims)[::-1]
        i = 0
        sim1 = []
        while sims[argsort[i]] == 1:
            sim1.append(hrefs_good[argsort[i]])
            i += 1
        eds = [editdistance.eval(target, s) for s in sim1]
        argsort[:len(sim1)] = [argsort[:len(sim1)][i] for i in np.argsort(eds)]
        topK = argsort[:top]

        if top > 1:
            print([hrefs_good[i] for i in topK])
        idx = int(input()) if top > 1 else 0
        assert idx in [0, 1, 2]
        now = hrefs_good[topK[idx]]
        print(unquote(now))
        path.append(now)
    return path

In [4]:
def run(start, target, similarity, top=1, middle=None):
    if middle is None:
        path = run_one(start, target, similarity, top)
        print('Done. Length = {}'.format(len(path)))
        return path
    else:
        middle = middle.lower()
        path1 = run_one(start, middle, similarity, top)
        path2 = run_one(middle, target, similarity, top)
        print('Done. Length = {}'.format(len(path1) + len(path2)))
        return path1 + path2

In [5]:
start = 'Fruit'
target = 'Hedgehog'.lower()

for similarity in ['fasttext', 'wordnet']:
    for middle in [None, 'United_States']:
        run(start, target, similarity=similarity, top=1, middle=middle)
        print('\n=====')

Squirrel
Mouse-like_hamster
Hamster
Roborovski_hamster
Rodent
Armadillo
Chipmunk
Red-tailed_chipmunk
Antelope_squirrel
Gray-footed_chipmunk
Townsend's_chipmunk
Lodgepole_chipmunk
Mammal
Rabbit
Rabbit_rabbit_rabbit
Three_hares
Rabbit_rabbit
Maneki-neko
Meowth
Blastoise
Greninja
Sonic_the_Hedgehog_(character)
Hedgehog
Done. Length = 24

=====
Supreme_Court_of_the_United_States
United_States
Helianthus
Helianthus_decapetalus
Muskrat
Mouse
Hamster
Mouse-like_hamster
Squirrel
Dormouse
Rodent
Armadillo
Chipmunk
Red-tailed_chipmunk
Antelope_squirrel
Gray-footed_chipmunk
Townsend's_chipmunk
Lodgepole_chipmunk
Mammal
Rabbit
Rabbit_rabbit_rabbit
Three_hares
Rabbit_rabbit
Maneki-neko
Meowth
Blastoise
Greninja
Sonic_the_Hedgehog_(character)
Hedgehog
Done. Length = 31

=====
Squirrel
New_World_porcupine
Andean_porcupine
Bahia_porcupine
Rothschild's_porcupine
Stump-tailed_porcupine
Bicolored-spined_porcupine
Prehensile-tailed_porcupine
Black-tailed_hairy_dwarf_porcupine
Porcupine
Hedgehog
Done. Leng

In [6]:
start = 'Fruit'
target = 'Machine_learning'.lower()

for similarity in ['fasttext', 'wordnet']:
    for middle in [None, 'United_States']:
        run(start, target, similarity=similarity, top=1, middle=middle)
        print('\n=====')

Wayback_Machine
WABAC_machine
ENIAC
Adding_machine
Electromechanical
Enigma_machine
C-36_(cipher_machine)
C-52_(cipher_machine)
Rotor_machine
Hebern_Rotor_Machine
Enigma_(machine)
Hebern_rotor_machine
Combined_Cipher_Machine
JADE_(cipher_machine)
Purple_(cipher_machine)
Enigma_Machine
NEMA_(machine)
Red_(cipher_machine)
Japanese_M-1_cipher_machine
International_Standard_Book_Number
COBOL
Mainframe_computer
Virtual_machine
Machine_learning
Done. Length = 25

=====
Supreme_Court_of_the_United_States
United_States
Machine_tool
Machine
Turing_machine
Post–Turing_machine
Computation
Computer
Machine_learning
Done. Length = 11

=====
Wayback_Machine
WABAC_machine
Plot_device
Stylistic_device
Rhetorical_device
Translation_(rhetorical_device)
Scientific_method
Inquiry-based_learning
Project-based_learning
Problem-based_learning
Phenomenon-based_learning
Deeper_learning
Hands-on_learning
Learning
Augmented_learning
Observational_learning
Machine_learning
Done. Length = 18

=====
Supreme_Court_o

In [7]:
def traverse(connect, current, lst):
    if current[-1] == len(lst)-1:
#         print(current)
        print(np.array(lst)[current])
        return
    next_ = list(current)
    next_.append(None)
    for i in connect[current[-1]]:
        next_[-1] = i
        traverse(connect, next_, lst)

def sublst(lst):
    connect = []
    for i in range(len(lst)):
        url = wiki_prefix + lst[i]
        res = requests.get(url)
        soup = BeautifulSoup(res.text, 'html.parser')
        body = soup.find('div', {'class': 'mw-parser-output'})
        reach = []
        for j in range(i+1, len(lst)):
            if body.find('a', {'href': '/wiki/' + lst[j]}) is not None:
                reach.append(j)
        connect.append(reach)
#     print(connect)
    traverse(connect, [0], lst)

In [8]:
path = run(start, target, similarity='wordnet', top=1)
sublst(path)

Wayback_Machine
WABAC_machine
Plot_device
Stylistic_device
Rhetorical_device
Translation_(rhetorical_device)
Scientific_method
Inquiry-based_learning
Project-based_learning
Problem-based_learning
Phenomenon-based_learning
Deeper_learning
Hands-on_learning
Learning
Augmented_learning
Observational_learning
Machine_learning
Done. Length = 18
['Fruit' 'Wayback_Machine' 'WABAC_machine' 'Plot_device'
 'Stylistic_device' 'Rhetorical_device' 'Translation_(rhetorical_device)'
 'Scientific_method' 'Inquiry-based_learning' 'Project-based_learning'
 'Problem-based_learning' 'Phenomenon-based_learning' 'Deeper_learning'
 'Hands-on_learning' 'Learning' 'Augmented_learning'
 'Observational_learning' 'Machine_learning']
['Fruit' 'Wayback_Machine' 'WABAC_machine' 'Plot_device'
 'Stylistic_device' 'Rhetorical_device' 'Translation_(rhetorical_device)'
 'Scientific_method' 'Inquiry-based_learning' 'Project-based_learning'
 'Problem-based_learning' 'Phenomenon-based_learning' 'Deeper_learning'
 'Hands-on_

In [9]:
def wn_path(w1, w2):
    s1 = wn.synsets(w1)[0]
    s2 = wn.synsets(w2)[0]
    lch = s1.lowest_common_hypernyms(s2)[0]
    
    path1 = s1.hypernym_paths()[0]
    path2 = s2.hypernym_paths()[0]
    
    i1 = path1.index(lch)
    i2 = path2.index(lch)
    
    return list(reversed(path2[i2:])) + path1[i1:]

In [10]:
hrefs = get_hrefs('fruit')

In [11]:
for synset in wn_path('fruit', 'hedgehog'):
    print(synset._name.split('.')[0])
    names = synset._name.split('.')[0].split('_')
    for href in hrefs:
        if all(name in href.lower() for name in names):
            print('---', href)

porcupine
rodent
placental
mammal
vertebrate
chordate
animal
--- Animal
--- Animal-free_agriculture
--- Animal_rights
--- Animal_welfare
--- Eating_Animals
organism
living_thing
whole
whole
natural_object
plant_part
plant_organ
reproductive_structure
fruit
--- Fruit_(disambiguation)
--- List_of_culinary_fruits
--- Orange_(fruit)
--- Nut_(fruit)
--- Fruit_anatomy
--- Accessory_fruit
--- Aggregate_fruit
--- Capsule_(fruit)
--- Follicle_(fruit)
--- Samara_(fruit)
--- Utricle_(fruit)
--- Multiple_fruit
--- Breadfruit
--- Kiwifruit
--- Grapefruit
--- Lime_(fruit)
--- Squash_(fruit)
--- Seedless_fruit
--- Fruit_beer
--- Fruit_Basket
--- Fruit_Bouquet
--- Ethylene-ripened_fruits
--- Fruit_tree
--- Fruitarianism
--- List_of_fruit_dishes
--- Fruit#Simple_fruit
--- Compound_fruit


In [12]:
run('animal', 'hedgehog', similarity='fasttext', top=1)
run('animal', 'hedgehog', similarity='wordnet', top=1)

Rabbit
Rabbit_rabbit_rabbit
Three_hares
Rabbit_rabbit
Maneki-neko
Meowth
Blastoise
Greninja
Sonic_the_Hedgehog_(character)
Hedgehog
Done. Length = 11
Mammal
Porcupine
Hedgehog
Done. Length = 4


['animal', 'Mammal', 'Porcupine', 'Hedgehog']